In [1]:
!pip install requests lxml


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import requests
import re
from lxml.html import fromstring, tostring, HtmlElement

Определим датафрейм `df` со столбцами "word" и "translation", в который будем записывать данные, предобрабатываемые на лету.

In [3]:
df = pd.DataFrame(columns=['word', 'translation'])

Определим общие функции обработки:

In [4]:
def _root_by_url(url: str) -> HtmlElement:
    """Возвращает объект распарсенной страницы, в котором можно искать элементы по XPATH."""
    return fromstring(requests.get(url).text)


def _add_record(word: str, translation: str) -> None:
    """Добавляет в конец датафрейма новую запись."""
    df.loc[len(df)] = (word, translation)

Парсинг статьи Fandom:

In [5]:
# Будем удалять из текста ненужные части.
DELETIONS = [
    '«',
    '»',
    'и т. п.',
    '(?)',
    'р. п.',
]

# Метки, указывающие, что текст не подходит как словарное слово.
STOP_SIGNS = [
    'фонетическое представление',
    'месяц в эльфском календаре',
    'артикль',
]

# Определим унифицированный разделитель слова и перевода.
SEP = '---SEP---'

root = _root_by_url('https://vedmak.fandom.com/wiki/Старшая_Речь/Лексика')
items = root.xpath('.//div[@id="mw-content-text"]/div[@class="mw-parser-output"]'
                   '/h2[contains(.,"Фразы и выражения")]/preceding-sibling::ul/li')

count = 0
for item in items:
    item = item.text_content().lower()

    # Проверяем текст на наличие меток о непригодности.
    stop = False
    for sign in STOP_SIGNS:
        if sign in item:
            stop = True
            break

    if stop:
        continue

    # Очищаем текст от лишних подстрок.
    for deletion in DELETIONS:
        item = item.replace(deletion, '')

    # После [ может следовать пояснение, убираем его.
    if '[' in item:
        item = item[:item.find('[')]

    # Устанавливаем общий разделитель (― и — не являются одним символом)
    item = item.replace('―', SEP).replace('—', SEP)

    # Разделителем синонимов может являться /, а не запятая. Сделаем запятую для унификации.
    item = item.replace('/', ',')

    # В конце, как правило, стоит точка. Она нам не нужна.
    item = item.rstrip('.')

    # Формируем всевозможные пары и заносим в датафрейм.
    words, translations = [x.strip().split(',') for x in item.split(SEP, maxsplit=1)]
    for word in words:
        word = word.strip()
        for translation in translations:
            translation = translation.strip()
            _add_record(word, translation)
            count += 1

print(f'В словарь добавлено {count} слов.')

В словарь добавлено 321 слов.


Парсинг статьи ВК:

In [6]:
# Будем удалять из текста ненужные части.
DELETIONS = [
    '1.',
    '2.',
    'эльф(-ы);',
    '(e)',
    '(-e)',
    '“',
    '”',
    ', т.е. объект или субъект, прин-й биосфере',

    '(для ровесников)',
    '(для группы)',
    '(при обращение к более старшему)',
    '.',
    '?',
    '!',
]

# Замены, упрощающие обработку некоторых моментов.
REPLACEMENTS = {
    'cead(-mil)': 'cead,ceadmil',
    '(также va faill)': ',va faill',
    'mint`и': 'mint`u',
    '-': ' ',
    '–': ' ',
    '–': ' ',
}

# Метки, указывающие, что текст не подходит как словарное слово.
STOP_SIGNS = [
    'глагол',
    'предлог',
    'частица',
    'артикль',
    'й месяц года',
]

# Здесь разделителем слова и перевода будет являться пробел, слева от которого - латинская буква, а справа - кириллическая.
SEP = r'(?<=[a-z])\s*(?=[а-я])'

root: HtmlElement = _root_by_url('https://vk.com/@thewhitewooolf-starshaya-rech-slovar')
ps: list[HtmlElement] = root.xpath('.//div[@id="article_view_-190268975_57657"]'
                                   '/h2[contains(.,"Выражения и фразы")]/preceding-sibling::p')

count = 0
for p in ps:
    # Заменим все <br> на переносы строк для удобства `str.split`
    for br in p.xpath('./br'):
        br.tail = "\n" + br.tail if br.tail else "\n"

    items = p.text_content().split('\n')
    for item in items:
        item = item.lower()

        # Проверяем текст на наличие меток о непригодности.
        stop = False
        for sign in STOP_SIGNS:
            if sign in item:
                stop = True
                break

        if stop:
            continue

        # Очищаем текст от лишних подстрок.
        for deletion in DELETIONS:
            item = item.replace(deletion, '')

        # Производим замены, облегчающие обработку:
        for from_, to in REPLACEMENTS.items():
            item = item.replace(from_, to)

        # После ( может следовать пояснение, убираем его.
        if '(' in item:
            item = item[:item.find('(')]

        # Также после [ может следовать пояснение, убираем его.
        if '[' in item:
            item = item[:item.find('[')]

        # Формируем всевозможные пары и заносим в датафрейм.
        words, translations = [x.strip().split(',') for x in re.split(SEP, item, maxsplit=1)]
        for word in words:
            word = word.strip()
            for translation in translations:
                translation = translation.strip()

                # В одном месте перепутали английскую и русскую "о".
                translation = translation.replace('o', 'о')

                _add_record(word, translation)
                count += 1

# ------ Дальше будем парсить отдельный специфичный блок внизу статьи ------

# Здесь разделитель тот же, но порядок обратный.
SEP_REVERSE = r'(?<=[а-я])\s*(?=[a-z])'
cur_sep = SEP_REVERSE

ps: list[HtmlElement] = root.xpath('.//div[@id="article_view_-190268975_57657"]'
                                   '/h2[contains(.,"РАЗГОВОРНИК ПОПУЛЯРНЫХ ФРАЗ")]/following-sibling::p')

for p in ps:
    # Заменим все <br> на переносы строк для удобства `str.split`
    for br in p.xpath('./br'):
        br.tail = "\n" + br.tail if br.tail else "\n"

    items = p.text_content().split('\n')
    for item in items:
        # Начиная с данного слова, снова начинаются обычные слова с обычным порядком, а не фразы. Делаем переход на лету.
        if 'Chodladh' in item:
            cur_sep = SEP

        item = item.lower()

        # Очищаем текст от лишних подстрок.
        for deletion in DELETIONS:
            item = item.replace(deletion, '')

        # Производим замены, облегчающие обработку:
        for from_, to in REPLACEMENTS.items():
            item = item.replace(from_, to)

        # Обрабатываем в зависимости от того фраза это или нет и заносим в датафрейм.
        if cur_sep == SEP:
            # Формируем всевозможные пары и заносим в датафрейм.
            words, translations = [x.strip().split(',') for x in re.split(SEP, item, maxsplit=1)]
            for word in words:
                word = word.strip()
                for translation in translations:
                    translation = translation.strip()
    
                    # В одном месте перепутали английскую и русскую "о".
                    translation = translation.replace('o', 'о')
    
                    _add_record(word, translation)
                    count += 1
        else:
            translation, word = [x.strip() for x in re.split(cur_sep, item, maxsplit=1)]
            _add_record(word, translation)
        count += 1

print(f'В словарь добавлено {count} слов.')

В словарь добавлено 932 слов.


Вывод результатов:

In [7]:
pd.set_option('display.max_rows', 10000)
df

,word,translation
0,aab,устье
1,aard,гора
2,aard,горный
3,aard,верхняя
4,aard,высший
5,aard,самый высокий
6,ard,гора
7,ard,горный
8,ard,верхняя
9,ard,высший


Мы собрали словарь из 1244 слов (устойчивых выражений, фраз).

В дальнейшем при реализации перевода планируется реализовать поиск по синонимам.
К примеру, если надо будет перевести слово "высочайший", которого в словаре нашем нет, то программа должна будет взять синоним, который уже у нас есть - "высший" (aard).

Если и в этом случае поиск будет неудачен, то будем делать перевод на английский/латинский.